This is a classification example to show how to train and test, in the context of AutoML, i.e., do pipeline selection on the training set and then evaluate the performance of the selected model on the test set.

In [1]:
# necessary modules
import sys
import pandas as pd
import os
import time
import numpy as np
import multiprocessing

# Our modules
automl_path = '../automl/'
sys.path.append(automl_path)
from auto_learner import AutoLearner
import util

#import scikit-learn modules
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# disable warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
#load and split dataset into training and test folds
data = load_iris()
x = np.array(data['data'])
y = np.array(data['target'])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# Example 1: a no-brainer use

In [3]:
# initialize the autolearner class
m = AutoLearner(p_type='classification', runtime_limit=100, load_imputed=True, verbose=False)

In [4]:
# fit autolearner on training set and record runtime
start = time.time()
m.fit(x_train, y_train)
elapsed_time = time.time() - start

In [5]:
# use the fitted autolearner for prediction on test set
y_predicted = m.predict(x_test)
print("prediction error: {}".format(util.error(y_test, y_predicted, 'classification')))    
print("elapsed time: {}".format(elapsed_time))

prediction error: 0.0
elapsed time: 42.08381485939026


In [6]:
# get names of the selected machine learning models
m.get_models()

{'ensemble method': 'select at most 5 pipelines with smallest cv error',
 'base learners': [{'imputer': {'algorithm': 'SimpleImputer',
    'hyperparameters': {'strategy': 'constant'}},
   'encoder': {'algorithm': None},
   'standardizer': {'algorithm': None},
   'dim_reducer': {'algorithm': None},
   'estimator': {'algorithm': 'KNN',
    'hyperparameters': {'n_neighbors': 11, 'p': 2}}},
  {'imputer': {'algorithm': 'SimpleImputer',
    'hyperparameters': {'strategy': 'constant'}},
   'encoder': {'algorithm': None},
   'standardizer': {'algorithm': None},
   'dim_reducer': {'algorithm': None},
   'estimator': {'algorithm': 'KNN',
    'hyperparameters': {'n_neighbors': 13, 'p': 1}}},
  {'imputer': {'algorithm': 'SimpleImputer',
    'hyperparameters': {'strategy': 'constant'}},
   'encoder': {'algorithm': None},
   'standardizer': {'algorithm': None},
   'dim_reducer': {'algorithm': None},
   'estimator': {'algorithm': 'KNN',
    'hyperparameters': {'n_neighbors': 5, 'p': 2}}},
  {'imputer

# Example 2: build an ensemble of models

In [7]:
#experimental settings
VERBOSE = False #whether to print out information indicating current fitting progress
N_CORES = 1 #number of cores
RUNTIME_BUDGET = 15

In [8]:
#autolearner arguments
autolearner_kwargs = {
    'p_type': 'classification',
    'runtime_limit': RUNTIME_BUDGET,
    'verbose': VERBOSE,
    'selection_method': 'min_variance',
    'stacking_alg': 'greedy',
    'n_cores': N_CORES,
    'build_ensemble': True,
}

In [9]:
#intialize the autolearner class
m = AutoLearner(**autolearner_kwargs)

In [10]:
# fit autolearner on training set and record runtime
start = time.time()
m.fit(x_train, y_train)
elapsed_time = time.time() - start

In [11]:
# use the fitted autolearner for prediction on test set
y_predicted = m.predict(x_test)
print("prediction error: {}".format(util.error(y_test, y_predicted, 'classification')))
print("elapsed time: {}".format(elapsed_time))

prediction error: 0.0514550264550265
elapsed time: 12.84459137916565


In [12]:
# get names of the selected pipelines
m.get_models()

{'ensemble method': 'select at most 5 pipelines with smallest cv error',
 'base learners': [{'imputer': {'algorithm': 'SimpleImputer',
    'hyperparameters': {'strategy': 'constant'}},
   'encoder': {'algorithm': None},
   'standardizer': {'algorithm': None},
   'dim_reducer': {'algorithm': None},
   'estimator': {'algorithm': 'KNN',
    'hyperparameters': {'n_neighbors': 11, 'p': 2}}},
  {'imputer': {'algorithm': 'SimpleImputer',
    'hyperparameters': {'strategy': 'most_frequent'}},
   'encoder': {'algorithm': None},
   'standardizer': {'algorithm': None},
   'dim_reducer': {'algorithm': None},
   'estimator': {'algorithm': 'KNN',
    'hyperparameters': {'n_neighbors': 1, 'p': 2}}},
  {'imputer': {'algorithm': 'SimpleImputer',
    'hyperparameters': {'strategy': 'most_frequent'}},
   'encoder': {'algorithm': None},
   'standardizer': {'algorithm': None},
   'dim_reducer': {'algorithm': None},
   'estimator': {'algorithm': 'KNN',
    'hyperparameters': {'n_neighbors': 1, 'p': 2}}},
  